In [ ]:
!git clone https://github.com/yuccadrew/sip.git

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('sip'))

import numpy as np
import scipy.sparse.linalg as spl

from sipmod import *

# Import mesh and compute basis functions
mesh_prefix = 'sip/docs/examples/meshes/mesh_ex01'
mesh = MeshTri.read(mesh_prefix+'.1', scale_factor=1e-6)
mesh.draw()